In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

# Import Flavonoids Dataset and Protease Inhibitor Drugs

In [2]:
fv_df = pd.read_csv("./data/flavonoids.csv")
ref_drug_df = pd.read_csv("./data/mpro-inhibitors.csv")

#print dimentions of both the datasets
print(fv_df.shape, ref_drug_df.shape)

(268, 6) (11, 6)


In [3]:
fv_df.head(3)

,smiles,compound_class,compound_subclass,name,molecular_weight,formula
0,COC1=CC(=CC(OC)=C1O)C1=[O+]C2=C(C=C1OC1OC(COC(...,Flavonoids,Anthocyanins,Malvidin 3-O-(6''-p-coumaroyl-glucoside),639.580,C32H31O14
1,CC(=O)OCC1OC(OC2=CC3=C(O)C=C(O)C=C3[O+]=C2C2=C...,Flavonoids,Anthocyanins,Delphinidin 3-O-(6''-acetyl-galactoside),507.421,C23H23O13
2,[H][C@]1(COC(C)=O)O[C@@]([H])(OC2=CC3=C(O)C=C(...,Flavonoids,Anthocyanins,Cyanidin 3-O-(6''-acetyl-galactoside),491.422,C23H23O12


In [4]:
ref_drug_df.head(3)

,smiles,compound_class,name,molecular_weight,formula,pubchem_id
0,CC(C)(C)NC(=O)C1CN(CCN1CC(CC(CC2=CC=CC=C2)C(=O...,protease inhibitor,Indinavir,613.8,C36H47N5O4,5362440
1,CC1=C(C(=CC=C1)C)OCC(=O)NC(CC2=CC=CC=C2)C(CC(C...,protease inhibitor,Lopinavir,628.8,C37H48N4O5,92727
2,CC(C)(C)C(C(=O)NC(CC1=CC=CC=C1)C(CN(CC2=CC=C(C...,protease inhibitor,Atazanavir,704.9,C38H52N6O7,148192


## Create a Ligand Dataset by combining dataframes

In [24]:
lig_df = fv_df[["smiles", "name", "molecular_weight", "formula"]]\
        .append(ref_drug_df[["smiles", "name", "molecular_weight", "formula"]], ignore_index=True)
lig_df.head()

,smiles,name,molecular_weight,formula
0,COC1=CC(=CC(OC)=C1O)C1=[O+]C2=C(C=C1OC1OC(COC(...,Malvidin 3-O-(6''-p-coumaroyl-glucoside),639.580,C32H31O14
1,CC(=O)OCC1OC(OC2=CC3=C(O)C=C(O)C=C3[O+]=C2C2=C...,Delphinidin 3-O-(6''-acetyl-galactoside),507.421,C23H23O13
2,[H][C@]1(COC(C)=O)O[C@@]([H])(OC2=CC3=C(O)C=C(...,Cyanidin 3-O-(6''-acetyl-galactoside),491.422,C23H23O12
3,OC[C@H]1O[C@@H](OC2=CC3=C(O)C=C(O)C=C3[O+]=C2C...,Cyanidin 3-O-galactoside,449.385,C21H21O11
4,OC[C@H]1O[C@@H](OC2=CC3=C(C=C(O)C=C3O)[O+]=C2C...,Cyanidin 3-O-glucoside,449.385,C21H21O11


## Create RDKit Molecule instance of the molecules
Create molecule from SMILES - (simplified molecular-input line-entry system )

In [48]:
# Create a list of RDKit Molecule Instance
mols = []
for _, row in lig_df.iterrows():
    m = Chem.MolFromSmiles(row.smiles)
    # Add Hydrogens
    m = Chem.AddHs(m)
    
    # Embed into 3D space and UFF optimize molecule
    AllChem.EmbedMolecule(m, AllChem.ETKDG())
    minimize_status = AllChem.UFFOptimizeMolecule(m, 2000)
    
    if not minimize_status == 0:
        print(f"Failed to minimize compound {row['name']}")
    
    # Compute Gasteiger Charges
    AllChem.ComputeGasteigerCharges(m)
    
    mols.append(m)

##  Calculate Molecular Descriptors

- Lipinski parameters
- LogP and MR using Crippen’s approach

In [56]:
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors

def descriptor_calculator(row):
    # row.name points to index of the row
    mol_idx = mols[row.name]
    row["logP"] = Descriptors.MolLogP(mol_idx)
    row['molecular_wt_calculated'] = Descriptors.MolWt(mol_idx)
    row['havy_atom_count'] = Descriptors.HeavyAtomCount(mol_idx)
    row["hydrogen_acceptors"] = Descriptors.NumHAcceptors(mol_idx)
    row["hydrogen_donors"] = Descriptors.NumHDonors(mol_idx)
    row["rotatable_bonds"] = Descriptors.NumRotatableBonds(mol_idx)
    row["amide_bonds"] = rdMolDescriptors.CalcNumAmideBonds(mol_idx)
    row["ring_count"] = Descriptors.RingCount(mol_idx)
    return row

lig_df_comp = lig_df.apply(descriptor_calculator, axis=1)

In [59]:
# Assign index column as LIG_ID column
lig_df_comp.index.name = "LIG_ID"

lig_df_comp.head()

,smiles,name,molecular_weight,formula,logP,molecular_wt_calculated,havy_atom_count,hydrogen_acceptors,hydrogen_donors,rotatable_bonds,amide_bonds,ring_count
LIG_ID,,,,,,,,,,,,
0,COC1=CC(=CC(OC)=C1O)C1=[O+]C2=C(C=C1OC1OC(COC(...,Malvidin 3-O-(6''-p-coumaroyl-glucoside),639.580,C32H31O14,2.6635,639.586,46,13,7,18,0,5
1,CC(=O)OCC1OC(OC2=CC3=C(O)C=C(O)C=C3[O+]=C2C2=C...,Delphinidin 3-O-(6''-acetyl-galactoside),507.421,C23H23O13,0.6584,507.424,36,12,8,14,0,4
2,[H][C@]1(COC(C)=O)O[C@@]([H])(OC2=CC3=C(O)C=C(...,Cyanidin 3-O-(6''-acetyl-galactoside),491.422,C23H23O12,0.9528,491.425,35,11,7,13,0,4
3,OC[C@H]1O[C@@H](OC2=CC3=C(O)C=C(O)C=C3[O+]=C2C...,Cyanidin 3-O-galactoside,449.385,C21H21O11,0.3820,449.388,32,10,8,12,0,4
4,OC[C@H]1O[C@@H](OC2=CC3=C(C=C(O)C=C3O)[O+]=C2C...,Cyanidin 3-O-glucoside,449.385,C21H21O11,0.3820,449.388,32,10,8,12,0,4


In [60]:
# Write CSV file of the descriptors
lig_df_comp.to_csv("./output/ligand_dataset.csv")

# Write Structures (3D) in a single SDF file containing Ligand Dataset

In [67]:
sdf_writer = Chem.SDWriter('./output/lig_dataset.sdf')
lig_properties = lig_df_comp.columns.to_list()
for i, mol in enumerate(mols):
    data_ref = lig_df_comp.iloc[i]
    mol.SetProp("LIG_ID", "%s"%i)
    mol.SetProp("_Name", data_ref['name'])
    for p in lig_properties:
         mol.SetProp(f"_{p}", str(data_ref[p]))
    sdf_writer.write(mol)
sdf_writer.close()

# Split and Convert the ligand dataset to Mol2

In [77]:
from openbabel import pybel

In [79]:
for mol in pybel.readfile("sdf", "./output/lig_dataset.sdf"):
    #print(mol.data['LIG_ID'])
    mol.write("mol2", "./output/ligands/%s.mol2" % mol.data['LIG_ID'])